In [ ]:
!wget https://machinehack-be.s3.amazonaws.com/shell_hackathon_to_protect_against_cyber_threats/Participants_Dataset_SHPACT.zip

--2023-10-26 20:11:49--  https://machinehack-be.s3.amazonaws.com/shell_hackathon_to_protect_against_cyber_threats/Participants_Dataset_SHPACT.zip
Resolving machinehack-be.s3.amazonaws.com (machinehack-be.s3.amazonaws.com)... 16.12.36.51, 52.219.160.207, 52.219.62.68, ...
Connecting to machinehack-be.s3.amazonaws.com (machinehack-be.s3.amazonaws.com)|16.12.36.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 327867 (320K) [application/zip]
Saving to: ‘Participants_Dataset_SHPACT.zip’

Participants_Datase 100%[===================>] 320.18K   352KB/s    in 0.9s    

2023-10-26 20:11:51 (352 KB/s) - ‘Participants_Dataset_SHPACT.zip’ saved [327867/327867]



In [ ]:
!unzip Participants_Dataset_SHPACT.zip

Archive:  Participants_Dataset_SHPACT.zip
  inflating: datafinal.json          
  inflating: sample_solution_submission.ipynb  
  inflating: submission.csv          
  inflating: __MACOSX/._submission.csv  


In [ ]:
import logging
import json
import requests
import openai
import pandas as pd
import concurrent.futures
from concurrent.futures import as_completed, ThreadPoolExecutor, ProcessPoolExecutor
import logging
import time

#Data

In [ ]:
df = pd.read_json("datafinal.json")
df.to_csv('train_shelll.csv',index=False)

In [ ]:
df.head(5)

,ID,Text,ContainsCode,CodeList
0,1,Test-driven development (TDD) is a software de...,,
1,2,Software development is an exciting field that...,False,
2,3,The development process often starts with a co...,,
3,4,Another important aspect of software developme...,True,git commit -m 'Initial commit'
4,5,"In the world of software development, language...",False,


In [ ]:
df['ContainsCode'].value_counts()

False    845
True     532
         531
Name: ContainsCode, dtype: int64

In [ ]:
res_set1= df[df['ContainsCode']==False]

In [ ]:
test_set1= df[df['ContainsCode']!=False]

In [ ]:
test_set1.shape,test_set1['ContainsCode'].value_counts()

((1063, 4),
 True    532
         531
 Name: ContainsCode, dtype: int64)

# Inferencing

In [ ]:
class GenOpenai(object):

  def __init__(self, openai_version: str):
    self._openai_version = openai_version
    self.set_conn_params(openai_version)

  @staticmethod
  def set_conn_params(openai_version):
    openai.api_type = OPENAI_TYPE
    openai.api_base = OPENAI_BASE
    openai.api_version = openai_version
    openai.api_key = OPENAI_KEY

  def get_api_message(self=None, system_message="You are an AI assistant that helps people find information.",
            user_message=None):
    return [{"role": "system", "content": system_message.strip()},
        {"role": "user", "content": user_message}]

  def show_conversation(self, messages):
    for message in messages:
      logging.info(f"[{message['role'].upper()}] : {message['content']}")

  def fetch_response(self, model, msg_prompt, parameters):
    self.set_conn_params(OPENAI_VERSION)
    try:
      response = openai.ChatCompletion.create(
        engine=model,
        messages=msg_prompt,
        temperature=parameters['temperature'],
        max_tokens=parameters['max_tokens'],
        top_p=parameters['top_p'],
        frequency_penalty=parameters['frequency_penalty'],
        presence_penalty=parameters['presence_penalty'],
      )
      return response

    except openai.error.APIError as e:
      # Handle API error here, e.g. retry or log
      logging.warning(f"OpenAI API returned an API Error: {e}")
      return None

    except openai.error.AuthenticationError as e:
      # Handle Authentication error here, e.g. invalid API key
      logging.warning(f"OpenAI API returned an Authentication Error: {e}")
      return None

    except openai.error.APIConnectionError as e:
      # Handle connection error here
      logging.warning(f"Failed to connect to OpenAI API: {e}")
      return None

    except openai.error.InvalidRequestError as e:
      # Handle connection error here
      logging.warning(f"Invalid Request Error: {e}")
      return None

    except openai.error.RateLimitError as e:
      # Handle rate limit error
      logging.warning(f"OpenAI API request exceeded rate limit: {e}")
      return None

    except openai.error.ServiceUnavailableError as e:
      # Handle Service Unavailable error
      logging.warning(f"Service Unavailable: {e}")
      return None

    except openai.error.Timeout as e:
      # Handle request timeout
      logging.warning(f"Request timed out: {e}")
      return None

    except:
      # Handles all other exceptions
      logging.warning("An exception has occured.")
      return None

In [ ]:
SLEEP_TIME = 0.75
TIMEOUT = 10
MAX_RETRY = 5


def concurrent_fetch_gpt_response(model, msg_prompt, parameters):
  retries = 0
  while retries < MAX_RETRY:
    with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
      future = executor.submit(genai_class.fetch_response, model, msg_prompt, parameters)
      try:
        result = future.result(timeout=TIMEOUT)
        if result:
          return result
        else:
          retries += 1
      except concurrent.futures.TimeoutError:
        logging.warning(f"API call timed out!!, retrying")
        retries += 1
  logging.error(f"{id} - Max retries reached, exiting!!")
  return None

In [ ]:
#use own credential for infrencing
OPENAI_TYPE = "azure"
OPENAI_BASE = "###"
OPENAI_VERSION = "2023-07-01-preview"
OPENAI_KEY = '###'
DEPLOYMENT_NAME = 'GPT4'
model = DEPLOYMENT_NAME

parameters = {
    'temperature': 0.1,
    'max_tokens': 50,
    'top_p': 0.2,
    'frequency_penalty': 0,
    'presence_penalty': 0
  }

genai_class = GenOpenai(OPENAI_VERSION)

gpt_resp=[]
for i in test_set1['Text']
  prompt = f'In the below prompt please extract the programming related coding structure only. If you are not getting any coding structure then response ""NO"" only. Prompt: {i}'
  api_prompt = genai_class.get_api_message(user_message=prompt)
  response = concurrent_fetch_gpt_response(model, api_prompt, parameters)
  cleaned_resp = response['choices'][0]['message']['content']
  if cleaned_resp == "NO":
    cleaned_resp = ""
  gpt_resp.append(cleaned_resp)

# result cleanup

In [ ]:
test_set1['response']=gpt_resp

<ipython-input-43-d68353f573ea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set1['response']=gpt_resp


In [ ]:
test_set1.shape

(1063, 6)

In [ ]:
res_df=pd.concat([test_set1,res_set1])
res_df.shape

(1908, 6)

In [ ]:
res_df = res_df.sort_values(by=["ID"])

In [ ]:
res_df['response']=res_df['response'].str.replace('\n', '').str.replace('\t', '')
res_df=res_df.fillna("")

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
s1 = res_df["response"]
t1 = mlb.fit_transform(s1)

In [ ]:
submission = pd.DataFrame(t1)
submission

,0,1,2,3,4,5,6,7,8,9,...,83,84,85,86,87,88,89,90,91,92
0,1,0,0,0,0,0,0,0,1,1,...,1,1,1,1,0,0,0,1,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,1,1,...,1,1,0,0,0,1,0,0,0,0
3,1,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1903,1,0,0,0,0,0,0,0,1,1,...,1,0,0,1,0,1,0,0,0,0
1904,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1905,1,1,0,0,0,0,0,1,1,1,...,1,1,0,1,0,0,0,1,0,1
1906,1,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,1,0,0,0,0


In [ ]:
submission.to_csv("finalls.csv",index=False)